# Voorspellen van politieke partij op basis van tekst

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd
import time
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import DutchStemmer
import sqlite3
import pandas as pd
from sklearn.cluster import KMeans

Inlezen SQL database

In [2]:
db = sqlite3.connect('reports.sqlite')

agenda_items = pd.read_sql_query("SELECT * FROM agenda_items", db)
speaker_items = pd.read_sql_query("SELECT * FROM speaker_turns", db)
speakers = pd.read_sql_query("SELECT * FROM speakers", db)

# controle of het inlezen goed gaat
agenda_items.head()
speaker_items.head()

,id,agenda_item,turn_no,speaker,txt
0,1,1,1,1,Detijdelijke voorzitter:Ik deel aan de Kamer m...
1,2,5,1,2,Voorzitter. Ik zal het idee van de heer Pechto...
2,3,5,2,2,Zo mogelijk in aanwezigheid van de minister-pr...
3,4,5,3,3,Ik kan mij die behoefte heel goed voorstellen....
4,5,5,4,4,Steun namens D66 voor het verzoek van de heer ...


Politieke partijen toevoegen aan personen wanneer deze bekend was
De personen die niet tot een politieke partij behoren zijn niet benoemd tot een politieke partij en deze zullen in de volgende stap (bij het mergen) vanzelf uit de dataset gehaald worden.

In [3]:
speakers.loc[speakers['id'] == 48,'political'] = 'VVD' # minister Rutte behoort tot de VVD
speakers.loc[speakers['id'] == 49,'political'] = 'CDA'# meneer van Dam behoort tot het CDA
speakers.loc[speakers['id'] == 51,'political'] = 'VVD' # minister Hennis-Plasschaert behoort bij de VVD
speakers.loc[speakers['id'] == 54,'political'] = 'PvdA' # staatssecretaris Dijksma behoort tot PvdA
speakers.loc[speakers['id'] == 56,'political'] = 'VVD' # staatssecretaris Wiebes behoort tot VVD
speakers.loc[speakers['id'] == 84,'political'] = 'VVD' # mevrouw Schippers behoort tot de VVD
speakers.loc[speakers['id'] == 94,'political'] = 'PvdA'# Staatssecretaris van Rijn behoort tot de PvdA
speakers.loc[speakers['id'] == 98,'political'] = 'PvdA'# minister Plasterk behoort tot de PvdA
speakers.loc[speakers['id'] == 106,'political'] = 'VVD' # minister Blok behoort tot de VVD
speakers.loc[speakers['id'] == 108,'political'] = 'PvdA' # minister Koenders behoort tot de PvdA
speakers.loc[speakers['id'] == 115,'political'] = 'PvdA' # minister Ploumen behoort tot de PvdA
speakers.loc[speakers['id'] == 118,'political'] = 'VVD' # minister Schultz van Haegen-Maas Geesteranus behoort tot de VVD
speakers.loc[speakers['id'] == 125,'political'] = 'PvdA' # minister Asscher behoort tot de PvdA
speakers.loc[speakers['id'] == 134,'political'] = 'PvdA' # straatssecretaris Kleijnsma behoort tot de PvdA
speakers.loc[speakers['id'] == 136,'political'] = 'PvdA' # minister Ploumen behoort tot de PvdA
speakers.loc[speakers['id'] == 140,'political'] = 'PvdA' # minister Dijsselbloem behoort tot de PvdA
speakers.loc[speakers['id'] == 141,'political'] = 'VVD' # staatssecretaris Dijkhoff behoort tot de VVD
speakers.loc[speakers['id'] == 148,'political'] = 'VVD' # minister Kamp behoort tot de VVD
speakers.loc[speakers['id'] == 149,'political'] = 'VVD' # staatssecretaris Dekker behoort tot de VVD
speakers.loc[speakers['id'] == 157,'political'] = 'PvdA' # minister Bussemaker behoort tot de PvdA
speakers.loc[speakers['id'] == 169,'political'] = 'SP' # mevrouw Leijten behoort tot de SP
speakers.loc[speakers['id'] == 180,'political'] = 'CU' # mevrouw Schouten behoort tot de CU
speakers.loc[speakers['id'] == 181,'political'] = 'VVD' # minister Bruins behoort tot de VVD
speakers.loc[speakers['id'] == 182,'political'] = 'CDA' # minister De Jonge behoort tot de CDA
speakers.loc[speakers['id'] == 183,'political'] = 'CDA' # minister Grapperhaus behoort tot de CDA
speakers.loc[speakers['id'] == 184,'political'] = 'D66' # minister Koolmees behoort tot de D66
speakers.loc[speakers['id'] == 185,'political'] = 'CDA' # minister Bijleveld behoort tot de CDA
speakers.loc[speakers['id'] == 186,'political'] = 'VVD' # minister Dekker behoort tot de VVD
speakers.loc[speakers['id'] == 189,'political'] = 'D66' # minister Kaag behoort tot de D66
speakers.loc[speakers['id'] == 190,'political'] = 'VVD' # minister Knops behoort tot de CDA
speakers.loc[speakers['id'] == 191,'political'] = 'VVD' # minister Zijlstra behoort tot de VVD
speakers.loc[speakers['id'] == 192,'political'] = 'CDA' # minister Hoekstra behoort tot de CDA
speakers.loc[speakers['id'] == 193,'political'] = 'D66' # Staatssecretaris Snel behoort tot de D66
speakers.loc[speakers['id'] == 194,'political'] = 'CU' # minister Slob behoort tot de CU
speakers.loc[speakers['id'] == 195,'political'] = 'VVD' # minister Wiebes behoort tot de VVD
speakers.loc[speakers['id'] == 197,'political'] = 'CDA' # staatssecretaris Keijzer behoort tot de CDA
speakers.loc[speakers['id'] == 198,'political'] = 'D66' # minister Ollogren behoort tot de D66
speakers.loc[speakers['id'] == 203,'political'] = 'VVD' # Staatssecretaris Visser behoort tot de VVD
speakers.loc[speakers['id'] == 204,'political'] = 'VVD' # Staatssecretaris Van Ark behoort tot de VVD
speakers.loc[speakers['id'] == 207,'political'] = 'D66' # Minister Van Engelshoven behoort tot de D66
speakers.loc[speakers['id'] == 210,'political'] = 'VVD' # Minister Van Nieuwenhuizen-Wijbenga behoort tot de VVD
speakers.loc[speakers['id'] == 211,'political'] = 'D66' # Staatssecretaris Van Veldhoven behoort tot de D66
speakers.loc[speakers['id'] == 212,'political'] = 'VVD' # Staatssecretaris Harbers behoort tot de VVD
speakers.loc[speakers['id'] == 213,'political'] = 'CU' # Staatssecretaris Blokhuis behoort tot de CU
speakers.loc[speakers['id'] == 217,'political'] = 'PvdD' # De heer Wassenberg behoort tot de PvdD
speakers.loc[speakers['id'] == 218,'political'] = 'VVD' # De heer Van Meenen behoort tot de D66
speakers.loc[speakers['id'] == 233,'political'] = 'CDA' # Minister Bijleveld-Schouten behoort tot de CDA
speakers.loc[speakers['id'] == 234,'political'] = 'D66' # Staatssecretaris Van Veldhoven-van der Meer behoort tot de D66

In [4]:
# controle van het aantal sprekers per politieke partij
speakers.groupby('political').count()

,id,prefix,name
political,,,
50PLUS,5,5,5
CDA,29,28,29
CU,3,3,3
ChristenUnie,6,6,6
D66,30,30,30
DENK,3,3,3
EP/CDA,1,1,1
EP/CU/SGP,1,1,1
EP/D66,1,1,1


Hieronder hebben we de sprekerdataset en wat ze gezegd hebben samengevoegd. Alle tekst van personen die Iedereen die niet tot een politieke partij behoord is uit de datset gehaald. 

In [5]:
merged = speaker_items.merge(speakers, left_on='speaker', right_on='id')[['txt','political']]

In [6]:
# controle welke partijen er zijn met hoeveel tekst
merged.groupby('political').count()

,txt
political,
50PLUS,3477
CDA,14344
CU,1863
ChristenUnie,3568
D66,14947
DENK,3773
EP/CDA,3
EP/CU/SGP,1
EP/D66,1


Uit bovenstaande lijst blijkt dat er door mensen van het europees parlement maar weinig wordt gezegd. Omdat hier zo weinig tekst van beschikbaar is, willen we deze niet meenemen in ons model, en hebben we alle teksten Europese parlementariers uit de dataset verwijderd.

In [7]:
merged = merged[~merged['political'].isin(['EP/CDA', 'EP/CU/SGP', 'EP/D66', 
                                           'EP/GroenLinks', 'EP/PvdA', 'EP/PvdD',
                                           'EP/SP', 'EP/VVD'])]
merged = merged.dropna()
merged = merged.replace('Forum voor Democratie','FvD')
merged = merged.replace('ChristenUnie','CU')
print(merged.groupby('political').count())

              txt
political        
50PLUS       3477
CDA         14344
CU           5431
D66         14947
DENK         3773
FvD          2095
GroenLinks  10093
PVV          8442
PvdA         8535
PvdD         4050
SGP          2720
SP          12083
VVD         20777


In [8]:
# split data in train en test set
X = merged['txt']
Y = merged.political.astype('category').values.codes
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

We willen ook onderzoeken of het helpt om een stemmer toe te voegen, op die manier worden vervoegingen van woorden als hetzelfde woord megeteld en we verwachten dat dit ons model beter zal laten presteren. 

In [9]:
# stemmer
stemmer = DutchStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

Hieronder hebben we een pijnlijn gebouwd om te classificeren. 
Aan deze pijplijn hebben we toegevoegd:
- een countvectorizer die telt hoe vaak woorden voorkomen in een tekst. Hieraan hebben we een stemmer toegevoegd die ervoor zorgd dat .... Om de stopwoorden te verwijderen konden 

In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), max_df = 0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())])
     
t = time.time()
pipeline.fit(X_train,Y_train)
predicted = pipeline.predict(X_test)
print(accuracy_score(predicted,Y_test))
print(time.time() - t)

# accuracy score: 0.20542565676627245

KeyboardInterrupt: 

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), max_df = 0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier())])
     
t = time.time()
pipeline.fit(X_train,Y_train)
predicted = pipeline.predict(X_test)
print(accuracy_score(predicted,Y_test))
print(time.time() - t)

# Accuracy score: 0.4075110589509795

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), max_df = 0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression())])
     
t = time.time()
pipeline.fit(X_train,Y_train)
predicted = pipeline.predict(X_test)
print(accuracy_score(predicted,Y_test))
print(time.time() - t)

# Accuracy score 0.4043062200956938

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), max_df = 0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC())])
     
t = time.time()
pipeline.fit(X_train,Y_train)
predicted = pipeline.predict(X_test)
print(accuracy_score(predicted,Y_test))
print(time.time() - t)

# duurt te lang om te runnen

In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), max_df = 0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', PassiveAggressiveClassifier())])
     
t = time.time()
pipeline.fit(X_train,Y_train)
predicted = pipeline.predict(X_test)
print(accuracy_score(predicted,Y_test))
print(time.time() - t)

#Accuracy score: 0.3936535162950257

C:\Users\corien\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in PassiveAggressiveClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.3936535162950257
192.40455865859985


In [11]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), max_df = 0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier())])
     
t = time.time()
pipeline.fit(X_train,Y_train)
predicted = pipeline.predict(X_test)
print(accuracy_score(predicted,Y_test))
print(time.time() - t)

#Accuracy score: 0.13099214588787578

0.13099214588787578
412.84075927734375


In [12]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), max_df = 0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', AdaBoostClassifier())])
     
t = time.time()
pipeline.fit(X_train,Y_train)
predicted = pipeline.predict(X_test)
print(accuracy_score(predicted,Y_test))
print(time.time() - t)

# Accuracy score: 0.252098943757335

0.252098943757335
302.8725302219391


In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), max_df = 0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', DecisionTreeClassifier())])
     
t = time.time()
pipeline.fit(X_train,Y_train)
predicted = pipeline.predict(X_test)
print(accuracy_score(predicted,Y_test))
print(time.time() - t)

# Accuracy score: 0.23431434503927057

0.23431434503927057
726.0362014770508


In [17]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), max_df = 0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())])
     
t = time.time()
pipeline.fit(X_train,Y_train)
predicted = pipeline.predict(X_test)
print(accuracy_score(predicted,Y_test))
print(time.time() - t)

# Accuracy score: 0.22808522163040534

C:\Users\corien\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.22808522163040534
516.2958481311798


Het best scorende model is de SDG classifier. We zullen nu (een deel) van de parameters proberen te optimaliseren, om het uiteindelijke model te bepalen. We hebben nu ook een andere manier gevonden om stopwoorden te verwijderen, namelijk met een lijst. We zullen eerst de parameters optimaliseren met een stopwoordenlijst en daarna nog een keer een optimalisatie maar dan door max_df op verschillende waarden te zetten.

In [10]:
from stop_words import get_stop_words
stopwoorden = get_stop_words('dutch')

In [13]:
# optimaliseren van de SGD Classifier met stopwoordenlijst
for data_verwijderen in [1,2,3]:
    print data_verwijderen



parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC(gamma="scale")
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(iris.data, iris.target)


pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), max_df = 0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier())])
     
t = time.time()
pipeline.fit(X_train,Y_train)
predicted = pipeline.predict(X_test)
print(accuracy_score(predicted,Y_test))
print(time.time() - t)

# nu met stop woorden

pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), stop_words = stopwoorden)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier())])
     
t = time.time()
pipeline.fit(X_train,Y_train)
predicted = pipeline.predict(X_test)
print(accuracy_score(predicted,Y_test))
print(time.time() - t)

0.4071048117721405
235.85338354110718


In [ ]:
for data_verwijderen in [1,2,3]:
    print data_verwijderen

In [ ]:
# voor als we dit nog nodig hebben
N = np.min(merged.groupby('political').size())

sample50PLUS = merged[merged['political'] == '50PLUS'].sample(n = N, random_state = 1)
sampleCDA = merged[merged['political'] == 'CDA'].sample(n = N, random_state = 1)
sampleCU = merged[merged['political'] == 'CU'].sample(n = N, random_state = 1)
sampleD66 = merged[merged['political'] == 'D66'].sample(n = N, random_state = 1)
sampleDENK = merged[merged['political'] == 'DENK'].sample(n = N, random_state = 1)
sampleFvD = merged[merged['political'] == 'FvD'].sample(n = N, random_state = 1)
sampleGL = merged[merged['political'] == 'GroenLinks'].sample(n = N, random_state = 1)
samplePVV = merged[merged['political'] == 'PVV'].sample(n = N, random_state = 1)
samplePvdA = merged[merged['political'] == 'PvdA'].sample(n = N, random_state = 1)
samplePvdD = merged[merged['political'] == 'PvdD'].sample(n = N, random_state = 1)
sampleSGP = merged[merged['political'] == 'SGP'].sample(n = N, random_state = 1)
sampleSP = merged[merged['political'] == 'SP'].sample(n = N, random_state = 1)
sampleVVD = merged[merged['political'] == 'VVD'].sample(n = N, random_state = 1)

data = pd.concat([sample50PLUS,sampleCDA,sampleCU,sampleD66,sampleDENK,sampleFvD,
                  sampleGL,samplePVV,samplePvdA,samplePvdD,sampleSGP,sampleSP,sampleVVD])

print(data.groupby('political').count())
data['political'] = data.political.astype('category').values.codes


# eventueel de nederlandse woordenlijst
# stopwoordenlijst inlezen
text_file = open("stopwoordenlijst.txt", "r")
stopwoorden = text_file.readlines()
stopwoorden = [line[:-1] for line in stopwoorden]
stopwoorden[0] = 'a'
print(stopwoorden)

In [15]:

stopwoorden

['aan',
 'al',
 'alles',
 'als',
 'altijd',
 'andere',
 'ben',
 'bij',
 'daar',
 'dan',
 'dat',
 'de',
 'der',
 'deze',
 'die',
 'dit',
 'doch',
 'doen',
 'door',
 'dus',
 'een',
 'eens',
 'en',
 'er',
 'ge',
 'geen',
 'geweest',
 'haar',
 'had',
 'heb',
 'hebben',
 'heeft',
 'hem',
 'het',
 'hier',
 'hij',
 'hoe',
 'hun',
 'iemand',
 'iets',
 'ik',
 'in',
 'is',
 'ja',
 'je',
 'kan',
 'kon',
 'kunnen',
 'maar',
 'me',
 'meer',
 'men',
 'met',
 'mij',
 'mijn',
 'moet',
 'na',
 'naar',
 'niet',
 'niets',
 'nog',
 'nu',
 'of',
 'om',
 'omdat',
 'onder',
 'ons',
 'ook',
 'op',
 'over',
 'reeds',
 'te',
 'tegen',
 'toch',
 'toen',
 'tot',
 'u',
 'uit',
 'uw',
 'van',
 'veel',
 'voor',
 'want',
 'waren',
 'was',
 'wat',
 'werd',
 'wezen',
 'wie',
 'wil',
 'worden',
 'wordt',
 'zal',
 'ze',
 'zelf',
 'zich',
 'zij',
 'zijn',
 'zo',
 'zonder',
 'zou']

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer = stemmed_words, ngram_range=(1,1), max_df = 0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', KMeans(n_clusters = 13, random_state=0, max_iter = 10))])
     
t = time.time()
merged['k_means_cluster'] = pipeline.fit_predict(X)
merged.head()
print(time.time() - t)

merged.head()

In [ ]:
print(merged.head())